In [11]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import ta  # Technical analysis library

# -----------------------------
# 1. File path handling and data loading
# -----------------------------
current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
data_dir = os.path.join(parent_dir, 'data')

# Example file from India folder
file_path = os.path.join(data_dir, 'India', 'finance_and_banking.csv')

# Load the data, handling the 2-row header
df = pd.read_csv(file_path, header=[0, 1])
df.columns = [f"{col[0]}.{col[1]}" if col[1] else col[0] for col in df.columns]
df = df.rename(columns={df.columns[0]: 'Date'})
df['Date'] = pd.to_datetime(df['Date'])
df.set_index('Date', inplace=True)

# Clean column names
df.columns = [col.split('.')[0] for col in df.columns]

# Convert to numeric
for col in df.columns:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# -----------------------------
# 2. Technical Indicators
# -----------------------------
# Bollinger Bands
df['bb_high'] = ta.volatility.bollinger_hband(df['Close'])
df['bb_low'] = ta.volatility.bollinger_lband(df['Close'])
df['bb_mavg'] = ta.volatility.bollinger_mavg(df['Close'])

# MACD
macd = ta.trend.MACD(df['Close'])
df['macd'] = macd.macd()
df['macd_signal'] = macd.macd_signal()
df['macd_diff'] = macd.macd_diff()

# RSI
df['rsi'] = ta.momentum.rsi(df['Close'])

# Ichimoku Cloud
ichimoku = ta.trend.IchimokuIndicator(high=df['High'], low=df['Low'])
df['ichimoku_a'] = ichimoku.ichimoku_a()
df['ichimoku_b'] = ichimoku.ichimoku_b()
df['ichimoku_base'] = ichimoku.ichimoku_base_line()
df['ichimoku_conversion'] = ichimoku.ichimoku_conversion_line()

# -----------------------------
# 3. Filter last 6 months
# -----------------------------
plot_df = df.last('6M')

# -----------------------------
# 4. Plotting Functions
# -----------------------------
plt.style.use("seaborn-v0_8-darkgrid")

def plot_bollinger(df, ticker):
    fig, ax = plt.subplots(figsize=(14,6))
    ax.set_title(f"Bollinger Bands", fontsize=14, weight="bold")
    
    ax.plot(df.index, df['Close'], label="Close Price", color="blue", lw=2)
    ax.plot(df.index, df['bb_high'], label="Upper Band", color="red", linestyle="--")
    ax.plot(df.index, df['bb_low'], label="Lower Band", color="green", linestyle="--")
    ax.plot(df.index, df['bb_mavg'], label="Moving Average", color="purple", linestyle=":")
    
    ax.legend(loc="upper left")
    plt.tight_layout()
    plt.savefig(f"{ticker}_bollinger.png")
    plt.close()

def plot_ichimoku(df, ticker):
    fig, ax = plt.subplots(figsize=(14,6))
    ax.set_title(f"Ichimoku Cloud", fontsize=14, weight="bold")
    
    ax.plot(df.index, df['Close'], label="Close Price", color="blue", lw=2)
    ax.plot(df.index, df['ichimoku_base'], label="Base Line", color="red", lw=1.5)
    ax.plot(df.index, df['ichimoku_conversion'], label="Conversion Line", color="orange", lw=1.5)
    
    # Cloud shading
    ax.fill_between(df.index, df['ichimoku_a'], df['ichimoku_b'], 
                    where=df['ichimoku_a'] >= df['ichimoku_b'], 
                    color="lightgreen", alpha=0.4, label="Bullish Cloud")
    ax.fill_between(df.index, df['ichimoku_a'], df['ichimoku_b'], 
                    where=df['ichimoku_a'] < df['ichimoku_b'], 
                    color="lightcoral", alpha=0.4, label="Bearish Cloud")
    
    ax.legend(loc="upper left")
    plt.tight_layout()
    plt.savefig(f"{ticker}_ichimoku.png")
    plt.close()

def plot_macd(df, ticker):
    fig, ax = plt.subplots(figsize=(14,4))
    ax.set_title(f"MACD", fontsize=14, weight="bold")
    
    ax.plot(df.index, df['macd'], label="MACD", color="blue", lw=2)
    ax.plot(df.index, df['macd_signal'], label="Signal", color="orange", lw=2)
    ax.bar(df.index, df['macd_diff'], color="gray", alpha=0.5, label="Histogram")
    ax.axhline(0, color="black", linestyle="--", alpha=0.7)
    
    ax.legend(loc="upper left")
    plt.tight_layout()
    plt.savefig(f"{ticker}_macd.png")
    plt.close()

def plot_rsi(df, ticker):
    fig, ax = plt.subplots(figsize=(14,4))
    ax.set_title(f"RSI (14 periods)", fontsize=14, weight="bold")
    
    ax.plot(df.index, df['rsi'], label="RSI", color="purple", lw=2)
    ax.axhline(70, color="red", linestyle="--", label="Overbought")
    ax.axhline(30, color="green", linestyle="--", label="Oversold")
    ax.set_ylim(0, 100)
    
    ax.legend(loc="upper left")
    plt.tight_layout()
    plt.savefig(f"rsi.png")
    plt.close()

# -----------------------------
# 5. Run all plots
# -----------------------------
ticker = "0P0001784S"
plot_bollinger(plot_df, ticker)
plot_ichimoku(plot_df, ticker)
plot_macd(plot_df, ticker)
plot_rsi(plot_df, ticker)

print("✅ Bollinger, Ichimoku, MACD, and RSI plots saved as separate PNG files.")


/tmp/ipykernel_29213/766983733.py:57: FutureWarning: last is deprecated and will be removed in a future version. Please create a mask and filter using `.loc` instead
  plot_df = df.last('6M')
/tmp/ipykernel_29213/766983733.py:57: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  plot_df = df.last('6M')


✅ Bollinger, Ichimoku, MACD, and RSI plots saved as separate PNG files.
